In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime
import time
from tqdm import tqdm
import sys
import glob

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

# Submited OOF CV

In [77]:
base = utils.read_df_pkl('../input/base_term*0*')[[key, target]].set_index(key)
display(base.head())
pred_col = 'prediction'
lb3664 = utils.read_pkl_gzip('../ensemble/good_submit_ensemble/0215_083_stack_submit_OUT29-7593_CV3-61266_LB3-664.gz')[[key, pred_col]].set_index(key)
lb3667 = utils.read_pkl_gzip('../ensemble/good_submit_ensemble/0213_074_submit_base-stack-lgb8_rm_out-lgb1_no_out-0_clf-thres0.0082_CV3.6088898700520273_LB3.667.gz')[[key, pred_col]].set_index(key)
base['PP_LB3-664_CV3-61266_OutlierFit29-7593_clf-thres0-01'] = lb3664[pred_col]
base['PP_LB3-667_CV3-61100_OutlierFit0_clf-thres0-082'] = lb3667[pred_col]
utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_oof_set/0219_go_OOF_CV_Submited_2MODEL_STACK_With_Outlier_or_Without_{len(base.columns)-1}set.csv')
# base.to_csv(f'../kaggle_oof_set/0219_go_OOF_CV_Submited_2MODEL_STACK_With_Outlier_or_Without_{len(base.columns)-1}set.csv')
base.head()

100%|██████████| 3/3 [00:00<00:00, 57.30it/s]


,target
card_id,
C_ID_92a2005557,-0.820283
C_ID_3d0044924f,0.392913
C_ID_d639edf6cd,0.688056
C_ID_186d6a6901,0.142495
C_ID_cdbd2c0db2,-0.159749


,target,PP_LB3-664_CV3-61266_OutlierFit29-7593_clf-thres0-01,PP_LB3-667_CV3-61100_OutlierFit0_clf-thres0-082
card_id,,,
C_ID_92a2005557,-0.820283,-0.343964,-0.335063
C_ID_3d0044924f,0.392913,0.226026,0.374489
C_ID_d639edf6cd,0.688056,0.750429,0.824915
C_ID_186d6a6901,0.142495,0.193548,0.130264
C_ID_cdbd2c0db2,-0.159749,-0.258822,-0.190934


# Stack前のRremove Outlier Single Model OOF

In [9]:
#========================================================================
# Stack前のRremove Outlier Single Model OOF
#========================================================================
    
base = utils.read_df_pkl('../input/base_term*0*')[[key, target]].set_index(key)
display(base.head())

rm_out_list = glob.glob('../ensemble/rm_outlier_ensemble/*lgb*199710*.gz')
rm_out_list = glob.glob('../ensemble/rm_outlier_ensemble/tmp/*lgb*.gz')

for path in rm_out_list:
    filename = re.search(r'/([^/.]*).gz', path.replace('.', '-')).group(1)
    if filename.count('CV8'):
        continue
    cv = re.search(r'CV([^/.]*)_LB', filename).group(1)
    lb = re.search(r'LB([^/.]*)', filename).group(1)
    
    try:
        eset = re.search(r'ESET([^/.])_', filename).group(1)
    except AttributeError:
        eset = 99
        
    try:
        feat_num = re.search(r'01_([^/%d]*)feats', filename).group(1)
    except AttributeError:
        feat_num = 0

    try:
        valid_type = re.search(r'valid-([^/%d]*)_E', filename).group(1)
    except AttributeError:
        if filename.count('ods'):
            valid_type = 'ods'
        elif filename.count('pmo'):
            valid_type = 'pmo'
        elif filename.count('pm'):
            valid_type = 'pm'
        elif filename.count('term'):
            valid_type = 'term'
        else:
            valid_type = '0'
    
    tmp = utils.read_pkl_gzip(path)
    if 'pred_mean' in tmp.columns:
        tmp = tmp[[key, 'pred_mean']].set_index(key)
        pred_col = 'pred_mean'
    else:
        tmp = tmp[[key, 'prediction']].set_index(key)
        pred_col = 'prediction'
        
    if filename.count('huber'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-huber'
    elif filename.count('fair'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-fair'
    elif filename.count('mape'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-mape'
    else:
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}'
        
    if filename.count('ridge'):
        col_name = f'ridge_lgb_stack_{col_name}'
    
    print(col_name)
    base[col_name] = tmp[pred_col]
utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_0221_OOF/0221_go_OOF_LB3662_Without_Outlier_GROUP_Single_LGB_{len(base.columns)-1}set.csv')
# base.to_csv(f'../kaggle_oof_set/0219_go_OOF_CV_Without_Outlier_GROUP_Single_LGB_{len(base.columns)-1}set.csv')
base.head()    

100%|██████████| 3/3 [00:00<00:00, 56.12it/s]


,target
card_id,
C_ID_92a2005557,-0.820283
C_ID_3d0044924f,0.392913
C_ID_d639edf6cd,0.688056
C_ID_186d6a6901,0.142495
C_ID_cdbd2c0db2,-0.159749


CV1-5540_LB_set99_feat340_valid-pm
CV1-5520_LB_set3_feat290_valid-pm
CV1-5486_LB_set0_feat298_valid-ods
CV1-5520_LB_set0_feat235_valid-ods_obj-fair
CV1-5530_LB_set2_feat320_valid-ods


,target,CV1-5540_LB_set99_feat340_valid-pm,CV1-5520_LB_set3_feat290_valid-pm,CV1-5486_LB_set0_feat298_valid-ods,CV1-5520_LB_set0_feat235_valid-ods_obj-fair,CV1-5530_LB_set2_feat320_valid-ods
card_id,,,,,,
C_ID_92a2005557,-0.820283,-0.265328,-0.305220,-0.318620,-0.311772,-0.341538
C_ID_3d0044924f,0.392913,0.376686,0.446242,0.165438,0.303839,0.050486
C_ID_d639edf6cd,0.688056,0.716802,0.751869,0.995924,0.714139,0.632040
C_ID_186d6a6901,0.142495,0.245714,0.130344,0.220188,0.224406,0.254490
C_ID_cdbd2c0db2,-0.159749,-0.124312,-0.175082,-0.237574,-0.271558,-0.256391


# RidgeによるLGBのStacking OOF. Exclude Outlier

In [6]:
#========================================================================
# RidgeによるLGBのStacking OOF. Exclude Outlier
#========================================================================
    
base = utils.read_df_pkl('../input/base_term*0*')[[key, target]].set_index(key)
display(base.head())

# rm_out_list = glob.glob('../stack/*ridge*rm_out*.gz')
rm_out_list = ['../stack/dir_level2/0215_083_stack_ridge_set-rm_out_lgb20_NN0_other0_OUT0_CV1.5464357357745486_LB.gz']

for path in rm_out_list:
    filename = re.search(r'/([^/.]*).gz', path.replace('.', '-')).group(1)
    if filename.count('CV8'):
        continue
    cv = re.search(r'CV([^/.]*)_LB', filename).group(1)
    lb = re.search(r'LB([^/.]*)', filename).group(1)
    
    try:
        eset = re.search(r'ESET([^/.])_', filename).group(1)
    except AttributeError:
        eset = 99
        
    try:
        feat_num = re.search(r'01_([^/%d]*)feats', filename).group(1)
    except AttributeError:
        feat_num = 0

    try:
        valid_type = re.search(r'valid-([^/%d]*)_E', filename).group(1)
    except AttributeError:
        if filename.count('ods'):
            valid_type = 'ods'
        elif filename.count('pmo'):
            valid_type = 'pmo'
        elif filename.count('pm'):
            valid_type = 'pm'
        elif filename.count('term'):
            valid_type = 'term'
        else:
            valid_type = '0'
    
    tmp = utils.read_pkl_gzip(path)
    if 'pred_mean' in tmp.columns:
        tmp = tmp[[key, 'pred_mean']].set_index(key)
        pred_col = 'pred_mean'
    else:
        tmp = tmp[[key, 'prediction']].set_index(key)
        pred_col = 'prediction'
        
    if filename.count('huber'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-huber'
    elif filename.count('fair'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-fair'
    elif filename.count('mape'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-mape'
    else:
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}'
        
    if filename.count('ridge'):
        col_name = f'ridge_lgb_stack_{col_name}'
    
    print(col_name)
    base[col_name] = tmp[pred_col]
# utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_oof_set/0219_go_OOF_CV_Without_Outlier_GROUP_Ridge_Stack_Some_LGB_{len(base.columns)-1}set.csv')
# base.to_csv(f'../kaggle_oof_set/0219_go_OOF_CV_Without_Outlier_GROUP_Ridge_Stack_Some_LGB_{len(base.columns)-1}set.csv')
base.head()    

100%|██████████| 3/3 [00:00<00:00, 54.18it/s]


,target
card_id,
C_ID_92a2005557,-0.820283
C_ID_3d0044924f,0.392913
C_ID_d639edf6cd,0.688056
C_ID_186d6a6901,0.142495
C_ID_cdbd2c0db2,-0.159749


ridge_lgb_stack_CV1-5464_LB_set99_feat0_valid-0


,target,ridge_lgb_stack_CV1-5464_LB_set99_feat0_valid-0
card_id,,
C_ID_92a2005557,-0.820283,-0.343964
C_ID_3d0044924f,0.392913,0.226026
C_ID_d639edf6cd,0.688056,0.750429
C_ID_186d6a6901,0.142495,0.193548
C_ID_cdbd2c0db2,-0.159749,-0.258822


# RidgeによるLGBのStacking OOF. All GROUP Dataset

In [8]:
#========================================================================
# RidgeによるLGBのStacking OOF. All GROUP Dataset
#========================================================================
base = utils.read_df_pkl('../input/base_term*0*')[[key, target]].set_index(key)
display(base.head())

oof_path_list = glob.glob('../model/LB3664_set/*ridge*.gz')
# oof_path_list = glob.glob('../stack/*ridge*set-all*.gz')

for path in oof_path_list:
    filename = re.search(r'/([^/.]*).gz', path.replace('.', '-')).group(1)
    cv = re.search(r'CV([^/.]*)_LB', filename).group(1)
    lb = re.search(r'LB([^/.]*)', filename).group(1)
    
    try:
        eset = re.search(r'ESET([^/.])_', filename).group(1)
    except AttributeError:
        eset = 99
        
    try:
        feat_num = re.search(r'01_([^/%d]*)feats', filename).group(1)
    except AttributeError:
        feat_num = 0

    try:
        valid_type = re.search(r'valid-([^/%d]*)_', filename).group(1)
    except AttributeError:
        if filename.count('ods'):
            valid_type = 'ods'
        elif filename.count('pmo'):
            valid_type = 'pmo'
        elif filename.count('pm'):
            valid_type = 'pm'
        else:
            valid_type = '0'
    
    tmp = utils.read_pkl_gzip(path)
    if 'pred_mean' in tmp.columns:
        tmp = tmp[[key, 'pred_mean']].set_index(key)
        pred_col = 'pred_mean'
    else:
        tmp = tmp[[key, 'prediction']].set_index(key)
        pred_col = 'prediction'
        
    if filename.count('huber'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-huber'
    elif filename.count('fair'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-fair'
    elif filename.count('mape'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-mape'
    else:
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}'
        
    if filename.count('ridge'):
        col_name = f'ridge_lgb_stack_{col_name}'
    
    print(col_name)
    base[col_name] = tmp[pred_col]
# utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_oof_set/0219_go_OOF_CV_Ridge_Stack_Some_LGB_{len(base.columns)-1}set.csv')
# base.to_csv(f'../kaggle_oof_set/0219_go_OOF_CV_Ridge_Stack_Some_LGB_{len(base.columns)-1}set.csv')
base.head()

100%|██████████| 3/3 [00:00<00:00, 54.67it/s]


,target
card_id,
C_ID_92a2005557,-0.820283
C_ID_3d0044924f,0.392913
C_ID_d639edf6cd,0.688056
C_ID_186d6a6901,0.142495
C_ID_cdbd2c0db2,-0.159749


ridge_lgb_stack_CV3-6184_LB_set99_feat0_valid-0


,target,ridge_lgb_stack_CV3-6184_LB_set99_feat0_valid-0
card_id,,
C_ID_92a2005557,-0.820283,-0.318154
C_ID_3d0044924f,0.392913,-0.419848
C_ID_d639edf6cd,0.688056,0.720410
C_ID_186d6a6901,0.142495,0.189931
C_ID_cdbd2c0db2,-0.159749,-0.059109


# Stacking前のLGB OOF CV Set

In [12]:
#========================================================================
# Stacking前のLGB OOF CV Set
base = utils.read_df_pkl('../input/base_term*0*')[[key, target]].set_index(key)
display(base.head())

# lgb_path_list = glob.glob('../model/LB3664_set/*.gz') + glob.glob('../ensemble/lgb_ensemble/*.gz')
lgb_path_list = glob.glob('../ensemble/dir_stack_blend/*.gz')

for path in lgb_path_list:
    
    filename = re.search(r'/([^/.]*).gz', path.replace('.', '-')).group(1)
    
#     if filename.count('NN') or filename.count('blend'):
#         continue
    if filename.count('NN'):
        continue
    
    cv = re.search(r'CV([^/.]*)_LB', filename).group(1)
    lb = re.search(r'LB([^/.]*)', filename).group(1)
    try:
        eset = re.search(r'ESET([^/.])_', filename).group(1)
    except AttributeError:
        eset = 99
    try:
        feat_num = re.search(r'01_([^/%d]*)feats', filename).group(1)
    except AttributeError:
        feat_num = 0
    try:
        valid_type = re.search(r'valid-([^/%d]*)_', filename).group(1)
    except AttributeError:
        if filename.count('ods'):
            valid_type = 'ods'
        elif filename.count('pmo'):
            valid_type = 'pmo'
        elif filename.count('pm'):
            valid_type = 'pm'
        else:
            valid_type = '0'
        
    
    tmp = utils.read_pkl_gzip(path)
    if 'pred_mean' in tmp.columns:
        tmp = tmp[[key, 'pred_mean']].set_index(key)
        pred_col = 'pred_mean'
    else:
        tmp = tmp[[key, 'prediction']].set_index(key)
        pred_col = 'prediction'
        
    if filename.count('huber'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-huber'
    elif filename.count('fair'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-fair'
    elif filename.count('mape'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-mape'
    else:
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}'
        
    print(col_name)
    base[col_name] = tmp[pred_col]

# utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_oof_set/0220_go_OOF_CV_single_LGB_{len(base.columns)-1}set.csv')
utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_0221_OOF/0221_go_OOF_LB3662_With_Outlier_Single_LGB_{len(base.columns)-1}set.csv')
# base.to_csv(f'../kaggle_oof_set/0219_go_OOF_CV_single_LGB_{len(base.columns)-1}set.csv')
base.head()
#========================================================================

100%|██████████| 3/3 [00:00<00:00, 60.22it/s]


,target
card_id,
C_ID_92a2005557,-0.820283
C_ID_3d0044924f,0.392913
C_ID_d639edf6cd,0.688056
C_ID_186d6a6901,0.142495
C_ID_cdbd2c0db2,-0.159749


CV3-6430_LB_set99_feat352_valid-0
CV3-6241_LB_set0_feat236_valid-ods
CV3-6416_LB_set2_feat320_valid-pm
CV3-6335_LB_set99_feat195_stack_lgb_lr0-01_271_valid-0
CV3-6335_LB_set99_feat173_valid-0
CV3-6216_LB_set99_feat296_valid-0
CV3-6453_LB_set1_feat340_valid-pm
CV3-6413_LB_set3_feat290_valid-ods
CV3-6248_LB_set0_feat235_valid-pm
CV3-6297_LB_set99_feat338_valid-0
CV3-6257_LB_set0_feat236_valid-ods
CV3-6402_LB_set2_feat320_valid-pmo
CV3-6221_LB_set0_feat235_valid-pmo
CV3-2381_LB_set99_feat272_valid-0
CV3-2400_LB_set99_feat261_valid-0
CV3-6431_LB_set99_feat303_valid-0
CV3-6174_LB_set0_feat288_valid-ods
CV3-6467_LB_set3_feat290_valid-pmo
CV3-6412_LB_set2_feat320_valid-ods
CV3-6437_LB_set3_feat290_valid-out
CV3-6401_LB_set1_feat340_valid-ods
CV3-6468_LB_set1_feat340_valid-pmo
CV3-6316_LB_set0_feat236_valid-pm
CV3-6450_LB_set1_feat340_valid-pm
CV3-6514_LB_set99_feat501_valid-0
CV3-6256_LB_set0_feat236_valid-pm
CV3-6246_LB_set99_feat295_valid-0
CV3-6263_LB_set99_feat191_valid-0
CV3-6311_LB_set9

,target,CV3-6430_LB_set99_feat352_valid-0,CV3-6241_LB_set0_feat236_valid-ods,CV3-6416_LB_set2_feat320_valid-pm,CV3-6335_LB_set99_feat195_stack_lgb_lr0-01_271_valid-0,CV3-6335_LB_set99_feat173_valid-0,CV3-6216_LB_set99_feat296_valid-0,CV3-6453_LB_set1_feat340_valid-pm,CV3-6413_LB_set3_feat290_valid-ods,CV3-6248_LB_set0_feat235_valid-pm,CV3-6297_LB_set99_feat338_valid-0,CV3-6257_LB_set0_feat236_valid-ods,CV3-6402_LB_set2_feat320_valid-pmo,CV3-6221_LB_set0_feat235_valid-pmo,CV3-2381_LB_set99_feat272_valid-0,CV3-2400_LB_set99_feat261_valid-0,CV3-6431_LB_set99_feat303_valid-0,CV3-6174_LB_set0_feat288_valid-ods,CV3-6467_LB_set3_feat290_valid-pmo,CV3-6412_LB_set2_feat320_valid-ods,CV3-6437_LB_set3_feat290_valid-out,CV3-6401_LB_set1_feat340_valid-ods,CV3-6468_LB_set1_feat340_valid-pmo,CV3-6316_LB_set0_feat236_valid-pm,CV3-6450_LB_set1_feat340_valid-pm,CV3-6514_LB_set99_feat501_valid-0,CV3-6256_LB_set0_feat236_valid-pm,CV3-6246_LB_set99_feat295_valid-0,CV3-6263_LB_set99_feat191_valid-0,CV3-6311_LB_set99_feat141_valid-0,CV3-6251_LB_set0_feat236_valid-pmo,CV3-6406_LB_set3_feat290_valid-ods,CV3-6196_LB_set99_feat260_valid-0,CV3-6266_LB_set99_feat191_valid-0,CV3-6275_LB3-675_set99_feat0_valid-0,CV3-6179_LB3-667_set0_feat298_valid-ods,CV3-6236_LB_set99_feat297_valid-0,CV3-6462_LB_set3_feat290_valid-pm,CV3-6333_LB_set99_feat0_valid-0,CV3-6226_LB_set0_feat235_valid-ods,CV3-6388_LB_set2_feat320_valid-ods,CV3-6536_LB_set99_feat306_valid-0
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,-0.820283,-0.533682,-0.389670,-0.310418,-0.292227,-0.321566,-0.349492,-0.204583,-0.293515,-0.346379,-0.346031,-0.367656,-0.316781,-0.360045,-0.403515,-0.344339,-0.262261,-0.342389,-0.360050,-0.284684,-0.325572,-0.151241,-0.229687,-0.481915,-0.188120,-0.257492,-0.405107,-0.348467,-0.270111,-0.293527,-0.373584,-0.324345,-0.337658,-0.367531,-0.349346,-0.360362,-0.344292,-0.375625,-0.331135,-0.340694,-0.332117,-0.181378
C_ID_3d0044924f,0.392913,-0.669546,-0.353189,-0.166621,-0.480098,-0.581695,-0.598341,-0.043758,-1.043000,-0.624106,-0.557220,-0.728224,-0.308836,-0.314517,-1.136101,-0.955046,-0.586575,-0.924811,-0.492303,-0.553925,-0.516587,-0.371366,-0.580510,-0.070213,-0.336599,-0.052061,-0.108786,-0.746501,-0.567469,-0.322127,-0.309417,-0.706560,-0.471055,-0.547171,-0.627140,-0.560401,-0.349898,-0.909553,-0.484170,-0.675536,-0.643977,-0.652553
C_ID_d639edf6cd,0.688056,0.581285,0.749703,0.645878,0.865657,0.849526,0.655104,0.593162,0.564796,0.817565,0.915316,0.666341,0.586407,0.732969,0.744735,0.651148,0.610568,0.697131,0.670619,0.553163,0.692051,0.528412,0.643725,0.780100,0.634536,0.759758,0.795536,0.677786,0.697874,0.790826,0.571900,0.521493,0.722236,0.697877,0.648794,0.822205,0.803631,0.676389,0.642721,0.759212,0.518067,0.584152
C_ID_186d6a6901,0.142495,0.118323,0.182120,0.133039,0.068197,0.214690,0.147356,0.108424,0.008181,0.220217,0.122011,0.163632,0.143492,0.172772,0.132069,0.012830,0.040901,0.159377,-0.043269,0.165502,-0.085453,0.162732,0.157530,0.208539,0.147047,0.000617,0.176037,0.153740,0.133529,0.131490,0.150416,-0.005467,0.161512,0.169382,0.111918,0.143494,0.137893,-0.029571,0.108993,0.168531,0.188852,0.205327
C_ID_cdbd2c0db2,-0.159749,-0.282362,-0.113891,-0.279065,-0.137075,-0.137133,-0.079374,0.005718,-0.123009,-0.151363,-0.223672,-0.081562,-0.296002,-0.067805,-0.126540,-0.218226,-0.071952,-0.143816,-0.095347,-0.328470,-0.146626,-0.019034,-0.021277,-0.057492,0.014068,-0.180205,-0.076840,-0.078756,-0.186935,-0.224225,-0.116971,-0.112846,-0.120586,-0.133997,-0.182233,-0.142937,-0.120741,-0.136741,-0.106126,-0.087869,-0.297711,-0.205778


## blend前の Clf OOF CV Set

In [6]:
#========================================================================
# Classifier
base = utils.read_df_pkl('../input/base_term*0*')[[key, target]].set_index(key)
display(base.head())

lgb_path_list = glob.glob('../stack/0207*binary*.gz')

for path in lgb_path_list:
    
    filename = re.search(r'/([^/.]*).gz', path.replace('.', '-')).group(1)
    
    if filename.count('NN') or filename.count('blend'):
        continue
    
    cv = re.search(r'CV([^/.]*)_', filename).group(1)
    try:
        eset = re.search(r'ESET([^/.])_', filename).group(1)
    except AttributeError:
        eset = 99
    try:
        feat_num = re.search(r'01_([^/%d]*)feats', filename).group(1)
    except AttributeError:
        feat_num = 0
    try:
        valid_type = re.search(r'valid-([^/%d]*)_', filename).group(1)
    except AttributeError:
        if filename.count('ods'):
            valid_type = 'ods'
        elif filename.count('pmo'):
            valid_type = 'pmo'
        elif filename.count('pm'):
            valid_type = 'pm'
        else:
            valid_type = '0'
        
    
    tmp = utils.read_pkl_gzip(path)
    if 'pred_mean' in tmp.columns:
        tmp = tmp[[key, 'pred_mean']].set_index(key)
        pred_col = 'pred_mean'
    else:
        tmp = tmp[[key, 'prediction']].set_index(key)
        pred_col = 'prediction'
        
    if filename.count('huber'):
        col_name = f'CV{cv[:6]}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-huber'
    elif filename.count('fair'):
        col_name = f'CV{cv[:6]}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-fair'
    elif filename.count('mape'):
        col_name = f'CV{cv[:6]}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-mape'
    else:
        col_name = f'CV{cv[:6]}_set{eset}_feat{feat_num}_valid-{valid_type}'
        
#     print(col_name)
    base[col_name] = tmp[pred_col]

utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_oof_set/0220_go_OOF_CV_Outlier_Clf_single_LGB_{len(base.columns)-1}set.csv')
# base.to_csv(f'../kaggle_oof_set/0219_go_OOF_CV_single_LGB_{len(base.columns)-1}set.csv')
base.head()
#========================================================================

100%|██████████| 3/3 [00:00<00:00, 42.47it/s]


,target
card_id,
C_ID_92a2005557,-0.820283
C_ID_3d0044924f,0.392913
C_ID_d639edf6cd,0.688056
C_ID_186d6a6901,0.142495
C_ID_cdbd2c0db2,-0.159749


,target,CV0-9099_set99_feat0_valid-0,CV0-9084_set99_feat0_valid-0
card_id,,,
C_ID_92a2005557,-0.820283,0.000426,0.000601
C_ID_3d0044924f,0.392913,0.008079,0.005600
C_ID_d639edf6cd,0.688056,0.004211,0.002847
C_ID_186d6a6901,0.142495,0.000789,0.000869
C_ID_cdbd2c0db2,-0.159749,0.000259,0.000187


### ~201901のLGB

In [20]:
#========================================================================
# Stacking前のLGB OOF CV Set
base = utils.read_df_pkl('../input/base_no_out_clf.gz')[[key, target]].set_index(key)
display(base.head())

lgb_path_list = glob.glob('../stack/dir_01/*.gz')

for path in lgb_path_list:
    
    filename = re.search(r'/([^/.]*).gz', path.replace('.', '-')).group(1)
    if filename.count('CV3-35'):
        print(filename)
    
    if filename.count('NN') or filename.count('blend'):
        continue
    
    cv = re.search(r'CV([^/.]*)_', filename).group(1)
    try:
        eset = re.search(r'ESET([^/.])_', filename).group(1)
    except AttributeError:
        eset = 99
    try:
        feat_num = re.search(r'01_([^/%d]*)feats', filename).group(1)
    except AttributeError:
        feat_num = 0
    try:
        valid_type = re.search(r'valid-([^/%d]*)_', filename).group(1)
    except AttributeError:
        if filename.count('ods'):
            valid_type = 'ods'
        elif filename.count('pmo'):
            valid_type = 'pmo'
        elif filename.count('pm'):
            valid_type = 'pm'
        else:
            valid_type = '0'
        
    
    tmp = utils.read_pkl_gzip(path)
    if 'pred_mean' in tmp.columns:
        tmp = tmp[[key, 'pred_mean']].set_index(key)
        pred_col = 'pred_mean'
    else:
        tmp = tmp[[key, 'prediction']].set_index(key)
        pred_col = 'prediction'
        
    if filename.count('huber'):
        col_name = f'CV{cv[:7]}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-huber'
    elif filename.count('fair'):
        col_name = f'CV{cv[:7]}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-fair'
    elif filename.count('mape'):
        col_name = f'CV{cv[:7]}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-mape'
    else:
        col_name = f'CV{cv[:7]}_set{eset}_feat{feat_num}_valid-{valid_type}'
        
    print(col_name)
    base[col_name] = tmp[pred_col]

utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_oof_set/0220_go_OOF_CV_single_LGB_Various_Validation_Set_Train_{len(base.columns)-1}set.csv')
# base.to_csv(f'../kaggle_oof_set/0219_go_OOF_CV_single_LGB_{len(base.columns)-1}set.csv')
base.head()
# ========================================================================

100%|██████████| 1/1 [00:00<00:00,  3.98it/s]


,target
card_id,
C_ID_92a2005557,-0.820283
C_ID_3d0044924f,0.392913
C_ID_d639edf6cd,0.688056
C_ID_186d6a6901,0.142495
C_ID_cdbd2c0db2,-0.159749


CV3-07015_set99_feat172_valid-pmo
CV5-24805_set99_feat172_valid-pm
CV3-31028_set99_feat172_valid-pmo
CV4-28736_set99_feat172_valid-pm
CV3-64902_set99_feat200_valid-0
CV8-15579_set99_feat172_valid-pmo
CV3-64871_set99_feat193_valid-0
CV3-64429_set99_feat172_valid-pm
CV3-64977_set99_feat186_valid-0
CV6-66725_set99_feat172_valid-pmo
CV3-64005_set99_feat172_valid-0
CV5-44443_set99_feat172_valid-pmo
CV7-74755_set99_feat172_valid-pm
CV3-64942_set99_feat188_valid-0
CV3-63678_set99_feat172_valid-0
CV3-38100_set99_feat172_valid-pmo
CV4-80832_set99_feat172_valid-pmo
CV7-65202_set99_feat172_valid-pm
CV5-58694_set99_feat172_valid-pm
CV4-43351_set99_feat172_valid-pmo
CV3-56606_set99_feat172_valid-pmo
CV3-65053_set99_feat177_valid-0
CV4-66190_set99_feat172_valid-pm
CV3-42651_set99_feat172_valid-pm
CV4-91818_set99_feat172_valid-ods
CV3-64031_set99_feat172_valid-0
CV3-25264_set99_feat172_valid-pm
CV3-73040_set99_feat172_valid-pmo
CV3-64887_set99_feat159_valid-0
CV4-41248_set99_feat172_valid-pmo
CV4-907

,target,CV3-07015_set99_feat172_valid-pmo,CV5-24805_set99_feat172_valid-pm,CV3-31028_set99_feat172_valid-pmo,CV4-28736_set99_feat172_valid-pm,CV3-64902_set99_feat200_valid-0,CV8-15579_set99_feat172_valid-pmo,CV3-64871_set99_feat193_valid-0,CV3-64429_set99_feat172_valid-pm,CV3-64977_set99_feat186_valid-0,CV6-66725_set99_feat172_valid-pmo,CV3-64005_set99_feat172_valid-0,CV5-44443_set99_feat172_valid-pmo,CV7-74755_set99_feat172_valid-pm,CV3-64942_set99_feat188_valid-0,CV3-63678_set99_feat172_valid-0,CV3-38100_set99_feat172_valid-pmo,CV4-80832_set99_feat172_valid-pmo,CV7-65202_set99_feat172_valid-pm,CV5-58694_set99_feat172_valid-pm,CV4-43351_set99_feat172_valid-pmo,CV3-56606_set99_feat172_valid-pmo,CV3-65053_set99_feat177_valid-0,CV4-66190_set99_feat172_valid-pm,CV3-42651_set99_feat172_valid-pm,CV4-91818_set99_feat172_valid-ods,CV3-64031_set99_feat172_valid-0,CV3-25264_set99_feat172_valid-pm,CV3-73040_set99_feat172_valid-pmo,CV3-64887_set99_feat159_valid-0,CV4-41248_set99_feat172_valid-pmo,CV4-90772_set99_feat172_valid-pm,CV3-64784_set99_feat209_valid-0,CV3-77163_set99_feat172_valid-pmo,CV3-80555_set99_feat172_valid-pmo,CV3-87660_set99_feat172_valid-pm,CV3-64972_set99_feat175_valid-0,CV4-12505_set99_feat172_valid-pm,CV4-08795_set99_feat172_valid-pm,CV3-64849_set99_feat182_valid-0,CV4-16515_set99_feat172_valid-pmo,CV7-91790_set99_feat172_valid-pmo,CV4-64219_set99_feat172_valid-pm,CV5-62703_set99_feat172_valid-pm,CV4-39195_set99_feat172_valid-pmo,CV3-63924_set99_feat172_valid-0,CV4-64215_set99_feat172_valid-pm,CV3-64904_set99_feat200_valid-0,CV3-42480_set99_feat172_valid-pm,CV3-22993_set99_feat172_valid-pmo,CV3-92925_set99_feat172_valid-pm,CV4-41717_set99_feat172_valid-pmo,CV7-52653_set99_feat172_valid-pmo,CV3-64951_set99_feat204_valid-0,CV3-64982_set99_feat175_valid-0,CV3-64926_set99_feat193_valid-0,CV5-41288_set99_feat172_valid-pm,CV3-54052_set99_feat172_valid-pmo,CV5-59204_set99_feat172_valid-pm,CV4-37218_set99_feat172_valid-pm,CV5-45854_set99_feat172_valid-pm,CV3-14081_set99_feat172_valid-pmo,CV3-89331_set99_feat172_valid-pm,CV3-65019_set99_feat180_valid-0,CV5-37798_set99_feat172_valid-pm,CV3-75348_set99_feat172_valid-pmo,CV5-52635_set99_feat172_valid-pmo,CV5-79755_set99_feat172_valid-pm,CV4-14460_set99_feat172_valid-pmo,CV3-64798_set99_feat190_valid-0,CV3-64642_set99_feat179_valid-0,CV4-47120_set99_feat172_valid-pm,CV4-11991_set99_feat172_valid-pmo,CV3-66376_set99_feat169_valid-0,CV5-92955_set99_feat172_valid-pm,CV3-20537_set99_feat172_valid-pm,CV4-66412_set99_feat172_valid-pm,CV3-64961_set99_feat169_valid-0,CV7-66402_set99_feat172_valid-pmo,CV3-64330_set99_feat180_valid-0,CV4-69120_set99_feat172_valid-pm,CV3-65623_set99_feat172_valid-pmo,CV3-96563_set99_feat172_valid-pm,CV3-64904_set99_feat205_valid-0,CV5-50175_set99_feat172_valid-pm,CV4-55551_set99_feat172_valid-pmo,CV3-27062_set99_feat172_valid-pmo,CV3-83461_set99_feat172_valid-pm,CV6-65360_set99_feat172_valid-pmo,CV4-91596_set99_feat172_valid-ods,CV3-13102_set99_feat172_valid-pm,CV5-45840_set99_feat172_valid-pmo,CV3-64114_set99_feat180_valid-0,CV3-64817_set99_feat184_valid-0,CV3-64961_set99_feat197_valid-0,CV3-65036_set99_feat192_valid-0,CV3-64091_set99_feat180_valid-0,CV4-57669_set99_feat172_valid-pm,CV3-18624_set99_feat172_valid-pm,CV5-49258_set99_feat172_valid-pm,CV6-80683_set99_feat172_valid-pmo,CV3-65171_set99_feat152_valid-0,CV3-73848_set99_feat172_valid-pmo,CV4-77547_set99_feat172_valid-pm,CV3-86836_set99_feat172_valid-pm,CV4-38035_set99_feat172_valid-pm,CV5-80228_set99_feat172_valid-pm,CV4-00942_set99_feat172_valid-pm,CV3-40511_set99_feat172_valid-pmo,CV3-71941_set99_feat172_valid-pmo,CV4-35107_set99_feat172_valid-pmo,CV3-64234_set99_feat180_valid-0,CV3-64976_set99_feat170_valid-0,CV3-64974_set99_feat175_valid-0,CV3-65038_set99_feat171_valid-0,CV4-08712_set99_feat172_valid-pmo,CV3-37315_set99_feat172_valid-pmo,CV3-35679_set99_feat172_valid-pmo,CV6-71242_set99_feat172_valid-pm,CV6-76957_set99_feat172_valid-pmo,CV3-64913_set99_feat200_valid-0,CV3-27036_set99_featstack

In [21]:
train = base[~base[target].isnull()]
# train[train.isnull()]
for col in train.columns:
    if train[col].isnull().sum()>0:
        print(col)

CV3-35304_set99_feat188_valid-0
CV3-35034_set99_feat200_valid-0
CV3-35023_set99_feat204_valid-0
